In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from yellowbrick.cluster import KElbowVisualizer

from scipy import stats 
from scipy.spatial.distance import mahalanobis
from scipy.cluster.hierarchy import dendrogram, fcluster

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import Ridge
from sklearn import metrics

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import jarque_bera

from pingouin import multivariate_normality

from sample_size_calculation import *

%load_ext autoreload
%autoreload 2

In [ ]:
data_to_cluster = pd.read_csv("Данные/processed/clustering-data-structure-volume.csv")
data_to_cluster.head()

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    return linkage_matrix

feature_labels_assets = ["Здания", "Машины и оборудование", \
    "Сооружения", "Транспортные средства"]
# feature_labels_assets = ["Здания", "Машины и оборудование", \
#     "Сооружения"]
X = data_to_cluster.loc[:, feature_labels_assets]
# scaler = MinMaxScaler()
# X["Всего основных фондов"] = scaler.fit_transform(X.loc[:, ["Всего основных фондов"]])

linkage_types = ['ward', 'complete', 'average', 'single']
cutoff_thresholds = [0.55, 0.5, 0.38, 0.25]
linkage_colors = dict(zip(linkage_types, cutoff_thresholds))
labels = [' Уорда', 'Полной связи', "Средней связи", "Ближнего соседа"]
linkage_labels = dict(zip(linkage_types, labels))

mpl.rcParams.update(mpl.rcParamsDefault)
hclust = dict()
fig, axs = plt.subplots(2, 2)
axs_indices = [(i, j) for i in range(2) for j in range(2)]
for linkage_type, axs_ind in zip(linkage_types, axs_indices):
    model = AgglomerativeClustering(linkage=linkage_type, distance_threshold=0, n_clusters=None)
    clustering = model.fit(X)
    axs[axs_ind].set_title(linkage_labels[linkage_type])
    axs[axs_ind].axhline(y=linkage_colors[linkage_type], linestyle='dashed', color='black')
    linkage_matrix = plot_dendrogram(
        clustering,
        truncate_mode="level",
        p=3,
        ax=axs[axs_ind], 
        labels=data_to_cluster["Код раздела"].to_list(),
        color_threshold = linkage_colors[linkage_type]
        # orientation='right'
        )
    
    hclust[linkage_type] = {
        "clustering": clustering,
        "linkage_matrix": linkage_matrix
    }
fig.tight_layout()
plt.show()


In [ ]:
chosen_x = [4., 4., 4., 3.]
chosen_y = [0.421457, 0.475347, 0.487812, 0.953896]
scores = []
fig, axs = plt.subplots(2, 2)
axs_indices = [(i, j) for i in range(2) for j in range(2)]
for i, linkage_type, axs_ind in zip(range(4), linkage_types, axs_indices):
    model = AgglomerativeClustering(linkage=linkage_type)
    visualizer = KElbowVisualizer(
        model,
        k=(1,15),
        ax=axs[axs_ind], 
        timings=False,
        title = linkage_labels[linkage_type],
        locate_elbow=True
        )
    visualizer.fit(X)
    visualizer.finalize()
    ticks = axs[axs_ind].get_xticks()
    ticks = np.append(ticks, chosen_x[i])
    axs[axs_ind].set_xticks(ticks)
    axs[axs_ind].plot([chosen_x[i]], [chosen_y[i]], \
        marker='o', ms=10, c='red')
    # if linkage_type == 'single':
    #     axs[axs_ind].axvline(x=linkage_chosen_k[linkage_type], color='red', \
    #         linestyle='dashdot')
    scores.append(visualizer.k_scores_)
fig.tight_layout()
# print()
plt.show()

In [ ]:
df = pd.DataFrame(scores)
df = df.transpose().rename(columns=dict(zip(df.columns, labels)))
df = df.rename(index=dict(zip(df.index, range(1,df.shape[0]))))
pd.set_option('display.precision', 2)
# pd.reset_option('display.precision')
df.head()

In [ ]:
cutoff_thresholds = cutoff_thresholds
cluster_mapping = data_to_cluster.loc[:, :"Код раздела"]
# cluster_mapping = data_to_cluster
cluser_count = []
for linkage_type, t in zip(linkage_types, cutoff_thresholds):
    enum = fcluster(hclust[linkage_type]["linkage_matrix"], t=t, criterion='distance')
    model = hclust[linkage_type]["clustering"]
    # hclust[linkage_type]["score"] = metrics.silhouette_score(X, enum, metric='euclidean')
    cluster_mapping[linkage_type] = enum
    cluser_count.append(cluster_mapping.nunique()[linkage_type])
# ind = data_to_cluster[data_to_cluster['Код раздела'].isin(['A','B'])].index
# data_to_cluster = data_to_cluster.drop(ind)
# cluster_mapping.head()
# data_to_cluster.head()
# cluser_count

In [ ]:
data_clustered = pd.merge(data_to_cluster, cluster_mapping, 'inner', "Код раздела")
data_clustered.to_csv('Данные/processed/clustering-mapping-structure.csv', index=False) 
data_clustered.head()

In [ ]:
clu_num = 4
pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
# data_clustered[data_clustered["ward"] == clu_num].loc[:, ["Название раздела"]]
data_clustered[data_clustered["ward"] == clu_num]

In [ ]:
data_for_regr = pd.read_csv('Данные/processed/regression-data-v0.csv')
feature_labels_douglas = ['ROFA', 'K','L', 't']
data_for_regr.head()

In [ ]:
X = data_for_regr.loc[:, "ROFA":"t"]
std_arr = X.std().to_numpy()
print(std_arr)
err_rate = 0.1
# delta_arr = X.mean().to_numpy() * err_rate
delta_arr = std_arr / 2.5
N = X.shape[0]
print(N)

sample_sizes = [sample_size_repetitive(0.05, delta, N, std=std) \
    for delta, std in zip(delta_arr, std_arr)]
sample_sizes

In [ ]:
data_for_regr[[*feature_labels_douglas]].corr()

In [ ]:
data_to_cluster = pd.read_csv("Данные/processed/regression-data-v0.csv")
data_clustered_full = pd.merge(data_to_cluster, cluster_mapping, 'inner', "Код раздела")
data_clustered_full = data_clustered_full.drop(columns=['Название раздела'])

In [ ]:
data_for_regr_cl = pd.merge(data_for_regr, cluster_mapping, 'inner', "Код раздела")
data_for_regr_cl.loc[:, "ROFA":"L"] = data_for_regr_cl.loc[:, "ROFA":"L"].apply(lambda x: np.log(x))
data_for_regr_cl = data_for_regr_cl.rename(columns={"ROFA": "ln_ROFA", "K": "ln_K", "L": "ln_L"})
data_for_regr_cl.to_csv("Данные/processed/regression-data-log-ward-v1.csv", index=False)
data_for_regr_cl.groupby('ward').count()

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')
# feature_labels_douglas_ln = ['ln_ROFA', 'ln_K', 'ln_L', 't']
# sns.pairplot(data_clustered_full.loc[:, ['Код раздела', *feature_labels_douglas]] \
#              [data_for_regr_cl['ward'] == 2], hue='Код раздела', corner=True)
# plt.show()

In [57]:
linkage_type = 'ward'
regr_metrics_all = []
for cluster_id in data_for_regr_cl[linkage_type].unique():
    cluster = data_for_regr_cl[data_for_regr_cl[linkage_type] == cluster_id]
    data_for_regr_cl[data_for_regr_cl['ward'] == cluster_id]

    regr_metrics = {}
    X = cluster.loc[:, ["ln_K", "ln_L", "t"]]
    X = sm.add_constant(X)
    Y = cluster.loc[:, ["ln_ROFA"]]
    regression = sm.OLS(Y, X, hasconst=True)
    # if cluster_id == 2:
    #     result_OLS = regression.fit_regularized(alpha=0, L1_wt=0)
    # else:
    result_OLS = regression.fit()

    n=X[X.columns[0]].count()
    regr_metrics['cluster_id'] = cluster_id
    regr_metrics['obs_count'] = n

    regr_metrics['shapiro-wilk'] = stats.shapiro(result_OLS.resid).pvalue
    regr_metrics['JB'] = jarque_bera(result_OLS.resid)[1]

    white_res = sm.stats.diagnostic.het_white(result_OLS.resid, regression.exog)
    white_lm_pvalue = white_res[1]
    regr_metrics['white_lm_pvalue'] = white_lm_pvalue

    dw = sm.stats.stattools.durbin_watson(result_OLS.resid)
    regr_metrics['D-W'] = dw

    VIF_array = []
    for exog_idx in range(1, regression.exog.shape[1]):
        VIF = variance_inflation_factor(regression.exog, exog_idx)
        VIF_array.append(round(VIF, 2))
        regr_metrics[f'VIF_x{exog_idx}'] = VIF
    # print('-- VIF:', VIF_array)
    hz, hz_pval, normal = multivariate_normality(regression.exog)
    regr_metrics['HZ'] = hz_pval
    regr_metrics_all.append(regr_metrics)

    # print('\nLinkage: {l}, cluster {c}, n: {n}'.format(l=linkage_type, c=cluster_id, n=n))
    # print(result_OLS.summary())    
    # print('-- White\'s test pvalue: ', white_lm_pvalue)
    # print('-- VIF:', VIF_array)

    # X_resid = X.iloc[:, 1:]
    # X_resid = X_resid.join(pd.DataFrame({"e": result_OLS.resid}))
    # X_resid = X_resid.join(data_for_regr_cl['Код раздела'])
    # g = sns.PairGrid(X_resid, y_vars=["e"], x_vars=["ln_K", "ln_L", "t"], hue='Код раздела', height=4)
    # # g.map(sns.regplot, color=".3", ci=None)
    # g.map(sns.scatterplot)
    # g.set(ylim=(-1.5, 1.5), yticks=[-1.5, -1, 0, 1, 1.5])

pd.reset_option('display.precision')
regr_metrics_all = pd.DataFrame(regr_metrics_all)
# print(regr_metrics_all)
regr_metrics_all[['shapiro-wilk', 'cluster_id']].sort_values(by='cluster_id') \
    .to_clipboard()
# plt.show()

# CHECK HETEROSCEDASTISITY

In [ ]:
# Check for multivariate normality
group_id = 3
d_clust_g3 = data_clustered_full[data_clustered_full['ward'] == group_id]

print(d_clust_g3.shape)
hz, pval, normal = multivariate_normality(d_clust_g3.loc[:, ['K', 'L', 't']])
print(normal)

In [ ]:
from help_funcs import isMahalanobisOutlier
is_outlier, squared_distances = isMahalanobisOutlier(d_clust_g3.loc[:, ['ROFA', 'K', 'L']]
                                  .to_numpy(), 0.55)
d_clust_g3['is_outlier'] = is_outlier
d_clust_g3['m_dist'] = squared_distances
d_clust_g3.loc[:, [ 'ROFA', 'K', 'L']]

# sns.histplot(d_clust_g3, x='m_dist')
# plt.show()
# sns.boxenplot(d_clust_g3, x='m_dist', k_depth='proportion', outlier_prop=0.5)
# plt.show()
# sns.boxenplot(d_clust_g3, x='m_dist', k_depth='trustworthy', trust_alpha=0.05)

g = sns.PairGrid(data=d_clust_g3.loc[:, ['ROFA', 'K', 'L']], corner=True)
g.map(sns.scatterplot, hue=d_clust_g3['Код раздела'], style=d_clust_g3['is_outlier'], 
      markers={True: 'X', False: 'o'})
g.add_legend()

In [ ]:
# SHUFFLE OUTLIERS
# Compute distortion first
from yellowbrick.cluster import distortion_score
data_shuffle1 = data_clustered_full.loc[:, ['Код раздела', *feature_labels_douglas, 'ward']].copy()
distortion1 = distortion_score(data_shuffle1[[*feature_labels_douglas]],
                               data_shuffle1['ward'])
data_shuffle2 = data_shuffle1.copy()
data_shuffle2.loc[data_shuffle2['Код раздела'] == 'G', 'ward'] = 4
distortion2 = distortion_score(data_shuffle2[[*feature_labels_douglas]],
                               data_shuffle2['ward'])
print(distortion1, distortion2)

data_shuffle2_cl4 = data_shuffle2[data_shuffle2['ward'] == 4]
g = sns.PairGrid(data=data_shuffle2_cl4.loc[:, ['ROFA', 'K', 'L']], corner=True)
g.map(sns.scatterplot, hue=data_shuffle2_cl4['Код раздела'],
      # style=data_shuffle2_cl4['is_outlier'], 
      markers={True: 'X', False: 'o'})
g.add_legend()

In [ ]:
# REMOVE OUTLIERS
dc = data_clustered_full.loc[:, [*feature_labels_douglas, 'ward']].copy()
centroids = dc.groupby('ward').mean()
to_move = data_clustered_full[data_clustered_full["Код раздела"] == 'G'].loc[:, \
    [*feature_labels_douglas]]
distances = centroids.apply(lambda c: np.linalg.norm(c - to_move.mean()),
    axis='columns')
distances
# cluster_mapping.loc[cluster_mapping['Код раздела'] == 'G', 'ward'] = 1
# cluster_mapping.loc[cluster_mapping['Код раздела'] == 'G', 'ward'] = 3

In [ ]:
# CORRECT HETEROSCEDASTISITY
from help_funcs import get_whites_regr
import warnings

linkage_type = 'ward'
# data_for_regr_cl = data_shuffle2.copy()
# data_for_regr_cl.loc[:, "ROFA":"L"] = data_for_regr_cl.loc[:, "ROFA":"L"].apply(lambda x: np.log(x))
# data_for_regr_cl = data_for_regr_cl.rename(columns={"ROFA": "ln_ROFA", "K": "ln_K", "L": "ln_L"})
regr_metrics_all = []

for cluster_id in [1, 4]:
    regr_metrics = {}

    cluster = data_for_regr_cl[data_for_regr_cl[linkage_type] == cluster_id]
    X = cluster.loc[:, ["ln_K", "ln_L", "t"]]
    X = sm.add_constant(X)
    Y = cluster.loc[:, ["ln_ROFA"]]
    regression = sm.OLS(Y, X, hasconst=True)
    result_OLS = regression.fit()


    n=X[X.columns[0]].count()
    for k in range(1, X.shape[1]):
        regr_white_WLS, whites_terms = get_whites_regr(regression, k)
        result_white_WLS = regr_white_WLS.fit()
        print('\nLinkage: {l}, cluster {c}, n: {n}'.format(l=linkage_type, c=cluster_id, n=n))
        print(f'White terms: {whites_terms}')
        print(result_white_WLS.summary())
        white_lm_pvalue = sm.stats.diagnostic.het_white(result_white_WLS.resid, X)[1]
        print('-- White\'s test pvalue: ', white_lm_pvalue)
    
    
    whites_regr, whites_terms = get_whites_regr(regression, full=True)
    result_white_WLS = whites_regr.fit()

    result_OLS = result_white_WLS
    # print('\nLinkage: {l}, cluster {c}, n: {n}'.format(l=linkage_type, c=cluster_id, n=n))
    # print(result_OLS.summary())    
    
    regr_metrics['cluster_id'] = cluster_id
    regr_metrics['obs_count'] = n

    white_lm_pvalue = sm.stats.diagnostic.het_white(result_OLS.resid, X)[1]
    print('-- White\'s test pvalue: ', white_lm_pvalue)
    regr_metrics['white_lm_pvalue'] = white_lm_pvalue

    # SSE_i = (result_OLS.resid ** 2).sum()
    # SSE_all.append(SSE_i)
    # print('-- SSE_i: ', round(SSE_i, 2))  

    for exog_idx in range(1, regression.exog.shape[1]):
        VIF = variance_inflation_factor(regression.exog, exog_idx)
        regr_metrics[f'VIF_x{exog_idx}'] = VIF
    # print('-- VIF:', VIF_array)
    regr_metrics['JB'] = jarque_bera(result_OLS.resid)[1]
    hz, hz_pval, normal = multivariate_normality(regression.exog)
    regr_metrics['HZ'] = hz_pval

    regr_metrics_all.append(regr_metrics)

    # X_resid = X_resid.join(data_for_regr_cl['Код раздела'])
    # g = sns.PairGrid(X_resid, y_vars=["e"], x_vars=["ln_K", "ln_L", "t"],
    #                  hue='Код раздела', height=4)
    # # g.map(sns.regplot, color=".3", ci=None)
    # g.map(sns.scatterplot)
    # g.set(ylim=(-1.5, 1.5), yticks=[-1.5, -1, 0, 1, 1.5])

pd.set_option('display.precision', 5)
pd.set_option('display.float_format', '{:,.5f}'.format)
regr_metrics_all = pd.DataFrame(regr_metrics_all)
# print(regr_metrics_all)
regr_metrics_all
# plt.show()

In [ ]:
DW_table = [ 
    ( 24, 0.881, 1.407 ),
    ( 18, 0.708, 1.422 ),
    ( 42, 1.15, 1.456 ),
]
DW_table = pd.DataFrame(DW_table, columns=['n', 'DW_L', 'DW_U'])
DW_table['4-DW_U'] = 4 - DW_table['DW_U']
DW_table['4-DW_L'] = 4 - DW_table['DW_L']
DW_table = DW_table.set_index('n', drop=False)
DW_table.loc[24, 'DW_L']

In [ ]:
# CORRECT AUTOREGRESSION
from help_funcs import ols_ar1, OLSAR1
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
from statsmodels.graphics.tsaplots import plot_acf

linkage_type = 'ward'
data_for_regr_cl = pd.merge(data_for_regr, cluster_mapping, 'inner', "Код раздела")
data_for_regr_cl.loc[:, "ROFA":"L"] = data_for_regr_cl.loc[:, "ROFA":"L"].apply(lambda x: np.log(x))
data_for_regr_cl = data_for_regr_cl.rename(columns={"ROFA": "ln_ROFA", "K": "ln_K", "L": "ln_L"})
SSE_all = []
# print(data_for_regr_cl[['ln_ROFA', 'ln_K', 'ln_L', 't']].corr())
regr_metrics_all = []
breush_godfrey_all = []
cochrane_log_all = {}

fig, axes = plt.subplots(2, 2)
fig.tight_layout()
axs_indices = [(i, j) for i in range(2) for j in range(2)]
cluster_ids = np.sort(data_for_regr_cl[linkage_type].unique())
for axs_ind, cluster_id in zip(axs_indices, cluster_ids):
    cluster = data_for_regr_cl[data_for_regr_cl[linkage_type] == cluster_id] \
        # .sort_values('t')

    regr_metrics = {}
    X = cluster.loc[:, ["ln_K", "ln_L", "t"]]
    X = sm.add_constant(X)
    Y = cluster.loc[:, ["ln_ROFA"]]
    regression = sm.OLS(Y, X, hasconst=True)
    result_OLS = regression.fit()

    plot_acf(result_OLS.resid, lags=10, ax=axes[axs_ind])
    axes[axs_ind].set_title(f'Кластер {cluster_id}')

    # RECURSIVE COCHRANE-ORCUTT with P-W fix
    def OLS_AR_n(original_model, max_lag, dw_vals=None, bg_alpha=0.05):
        dw_l, dw_u = dw_vals

        proc_log = {}

        AR_n1 = original_model
        first_lag = 2 + int(original_model.df_model)
        for n_lags in range(0, max_lag+1):
            e_1 = result_OLS.resid[1:].reset_index(drop=True)
            e_0 = result_OLS.resid[:-1].reset_index(drop=True)

            AR_n1_result = AR_n1.fit()
            dw_hat = durbin_watson(AR_n1_result.resid)

            if n_lags > 0:
                bg_result = acorr_breusch_godfrey(original_model.fit(), nlags=n_lags, store=True)
                bg_OLS_result = bg_result[4].resols
                bg_fpvalue = bg_result[3]

                tstat_pvals_all = stats.t.sf(bg_OLS_result.tvalues, bg_OLS_result.df_resid)
                tstat_pval_lag = tstat_pvals_all[first_lag + n_lags - 1]
            else:
                tstat_pval_lag = np.nan
                bg_fpvalue = np.nan

            proc_step_log = {
                # 'n lags' : n_lags,
                'DW-stat' : dw_hat,
                'BG t-stat p-value' : tstat_pval_lag,
                'BG F-stat p-value' : bg_fpvalue,
            }
            proc_log[n_lags] = proc_step_log

            aux_reg = sm.OLS(e_0, e_1)

            result_aux = aux_reg.fit()
            rho = result_aux.params[0]
            AR_n0 = ols_ar1(AR_n1, rho)
            AR_n1 = AR_n0
        return(AR_n1, proc_log)

    n=X[X.columns[0]].count()

    max_lag = 6
    dw_vals = (DW_table.loc[n, 'DW_L'], DW_table.loc[n, 'DW_U'])
    OLS_AR, proc_log = OLS_AR_n(regression, max_lag, dw_vals)
    cochrane_log_all[cluster_id] = pd.DataFrame(proc_log)
    result_OLS = OLS_AR.fit()
    print(result_OLS.summary())

    breush_godfrey = {}
    breush_godfrey['cluster_id'] = cluster_id
    for p in range(1, max_lag+1):
        fpval = sm.stats.diagnostic.acorr_breusch_godfrey(result_OLS, p)[1]
        breush_godfrey[f'p = {p}'] = fpval
    breush_godfrey_all.append(breush_godfrey)

    regr_metrics['cluster_id'] = cluster_id
    regr_metrics['n'] = n

    white_pvalue = sm.stats.diagnostic.het_white(result_OLS.resid, X)[1]
    regr_metrics['white_pvalue'] = white_pvalue

    regr_metrics['R^2'] = result_OLS.rsquared
    regr_metrics['f_pvalue'] = result_OLS.f_pvalue
    
    VIF_array = []
    for exog_idx in range(1, regression.exog.shape[1]):
        VIF = variance_inflation_factor(regression.exog, exog_idx)
        VIF_array.append(round(VIF, 2))
        regr_metrics[f'VIF_x{exog_idx}'] = VIF

    regr_metrics['JB'] = jarque_bera(result_OLS.resid)[1]
    hz, hz_pval, normal = multivariate_normality(regression.exog)

    regr_metrics['HZ'] = hz_pval
    regr_metrics_all.append(regr_metrics)

    dw = sm.stats.stattools.durbin_watson(result_OLS.resid)
    regr_metrics['DW'] = dw

    # print('\nLinkage: {l}, cluster {c}, n: {n}'.format(l=linkage_type, c=cluster_id, n=n))
    # print(result_OLS.summary())    
    # print('-- White\'s test pvalue: ', white_pvalue)
    # print('-- VIF:', VIF_array)

    # X_resid = X.iloc[:, 1:]
    # X_resid = X_resid.join(pd.DataFrame({"e": result_OLS.resid}))
    # X_resid = X_resid.join(data_for_regr_cl['Код раздела'])
    # g = sns.PairGrid(X_resid, y_vars=["e"], x_vars=["ln_K", "ln_L", "t"], hue='Код раздела', height=4)
    # # g.map(sns.regplot, color=".3", ci=None)
    # g.map(sns.scatterplot)
    # g.set(ylim=(-1.5, 1.5), yticks=[-1.5, -1, 0, 1, 1.5])

pd.set_option('display.float_format', lambda x: '%.2f' % x)
breush_godfrey_all = pd.DataFrame(breush_godfrey_all)
regr_metrics_all = pd.DataFrame(regr_metrics_all)
regr_metrics_acorr = pd.DataFrame(regr_metrics_all.loc[:, [ 'cluster_id', 'n', 'DW' ]]) \
    .merge(DW_table.reset_index(drop=True), on='n', how='left')

# cochrane_log_all = pd.DataFrame(cochrane_log_all)
cochrane_log_all = pd.concat(cochrane_log_all, keys=cochrane_log_all.keys())
cochrane_log_all = cochrane_log_all.rename_axis(['cluster_id', 'test'])
cochrane_log_all.sort_values(by='cluster_id').to_clipboard(float_format="%.2f", decimal=',')
# print(regr_metrics_all)

regr_metrics_acorr.sort_values('cluster_id', axis='index'). \
    style.format({
        'DW': '{:.2f}',
        'DW_L': '{:.2f}',
        'DW_U': '{:.2f}',
        '4-DW_U': '{:.2f}',
        '4-DW_L': '{:.2f}',
        }).hide(axis='index')

breush_godfrey_all.sort_values(by='cluster_id', axis='index').style.format('{:.2f}') \
    .format('{:d}', subset=['cluster_id']).hide(axis='index')
cochrane_log_all.sort_values(by='cluster_id')

In [ ]:
DW_table

In [ ]:
d_clust_g2 = data_for_regr_cl[data_for_regr_cl['ward'] == 2][
    ['ln_ROFA', 'ln_K', 'ln_L', 't']]
d_clust_g2.corr()
# sns.heatmap(d_clust_g2.corr(), fmt='.2f', annot=True)
# plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(d_clust_g2['ln_ROFA'], d_clust_g2['ln_K'], d_clust_g2['ln_L'])
plt.show()

In [ ]:
from scipy.stats import chi2
from matplotlib import cm
from matplotlib.colors import ListedColormap, Normalize

fig, ax = plt.subplots(1, 1)
df = 3
x = np.linspace(chi2.ppf(1e-10, df), chi2.ppf(0.999, df), 100)
rv = chi2(df)
d_clust_g3.loc[:, ['is_outlier', 'Код раздела']]
ax.plot(x, rv.pdf(x), c='black')

palette = sns.color_palette(n_colors=d_clust_g3['Код раздела'].nunique())
my_cmap = ListedColormap(palette.as_hex())
codes = d_clust_g3['Код раздела'].unique()
industry_colors = {code: (color if code == 'G' else 'grey') for code, color in zip(codes, my_cmap.colors)}
industry_linestyles = {code: ('dashed' if code == 'G' else 'solid') for code in codes}

# for x, ymax in zip(squared_distances, rv.pdf(squared_distances)):
for idx, obs in d_clust_g3.iterrows():
    ax.axvline(obs['m_dist'], ymax=rv.pdf(obs['m_dist'])*4.1,
               linestyle=industry_linestyles[obs['Код раздела']], c=industry_colors[obs['Код раздела']])

x2 = np.linspace(2.2, chi2.ppf(0.999, df), 80)
ax.fill_between(x2, rv.pdf(x2))
plt.margins(x=0, y=0)
fig.tight_layout()
plt.show()

In [ ]:
sns.pairplot(data_clustered_full.loc[:, 'ROFA':'ward'],
            corner=True, hue='ward', diag_kind='hist',
            palette=sns.color_palette(n_colors=cluser_count[0]))
plt.show()

In [ ]:
data_for_ANOVA = pd.merge(cluster_mapping, data_for_regr.loc[:, ["Код раздела", "ROFA"]], 'inner', 'Код раздела')
for linkage_type in linkage_types:
    groups = []
    for group_id in data_for_ANOVA[linkage_type].unique():
        groups.append(data_for_ANOVA["ROFA"][(data_for_ANOVA[linkage_type] == group_id)].to_numpy())
    res = stats.bartlett(*groups)
    print('bartlett', res.pvalue)
    res = stats.kruskal(*groups)
    print('kruskal', res.pvalue)